# Notebook Summary

This is Kaggle project aims to use EEG and Spectrogram data to classify harmful brain activity. More information about the competition and the data can be found [here](https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/overview).

# Load the Data

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

## Data Import

In [2]:
# Directories
DIR_INPUT = Path("/kaggle/input")
DIR_DATA = DIR_INPUT / "hms-harmful-brain-activity-classification"
DIR_EXAMPLES = DIR_DATA / "example_figures"
DIR_TEST_EEGS = DIR_DATA / "test_eegs"
DIR_TEST_SPECT = DIR_DATA / "test_spectrograms"
DIR_TRAIN_EEGS = DIR_DATA / "train_eegs"
DIR_TRAIN_SPECT = DIR_DATA / "train_spectrograms"

**train.csv:** Metadata for the train set. The expert annotators reviewed 50 second long EEG samples plus matched spectrograms covering 10 a minute window centered at the same time and labeled the central 10 seconds. Many of these samples overlapped and have been consolidated. train.csv provides the metadata that allows you to extract the original subsets that the raters annotated.

In [3]:
train = pd.read_csv(DIR_DATA / "train.csv")
train["eeg_id"]

Series([], Name: eeg_id, dtype: object)

In [4]:
# Write once error with train.csv is fixed
## Note: train.csv currently empty - competition creator is looking in to this issue
def get_eeg_spect_match():
    """Retrieves the annotated data for the given id."""
    pass